# Camada Ouro - Notebook único - Criação do Esquema SQL

In [0]:
### importes necessários

from pyspark.sql import SparkSession

## Passo 1 - Importar os dataframes da camada prata

In [0]:
# Caminho do arquivo Delta Lake
file_path = "/mnt/dados/camada_prata_terremotos"

# Ler o arquivo Delta Lake
df_terremotos = spark.read.format("delta").load(file_path)

df_terremotos.printSchema()
df_terremotos.show(10)

root
 |-- id: string (nullable = true)
 |-- prop_mag: double (nullable = true)
 |-- prop_url: string (nullable = true)
 |-- data: date (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pais: string (nullable = true)

+----------+--------+--------------------+----------+--------+---------+------------+
|        id|prop_mag|            prop_url|      data|latitude|longitude|        pais|
+----------+--------+--------------------+----------+--------+---------+------------+
|at00sthakw|     6.5|https://earthquak...|2025-03-21|     7.2|    -82.3|Desconhecido|
|us10001ldx|     6.2|https://earthquak...|2015-03-10|  6.7757| -72.9875|    Colombia|
|us10001pn4|     5.1|https://earthquak...|2015-03-22|   6.804|  -73.147|    Colombia|
|us10001rcr|     5.5|https://earthquak...|2015-03-27| -1.2012| -77.5836|     Ecuador|
|us10002azw|     5.1|https://earthquak...|2015-05-20| -3.0826| -77.5501|        Perú|
|us100030j8|     5.4|https://earthquak...

In [0]:
# Caminho do arquivo Delta Lake
file_path = "/mnt/dados/camada_prata_instalacoes"

# Ler o arquivo Delta Lake
df_instalacoes = spark.read.format("delta").load(file_path)

df_instalacoes.printSchema()
df_instalacoes.show(3)

root
 |-- codigo: string (nullable = true)
 |-- instalacao: string (nullable = true)
 |-- operador: string (nullable = true)
 |-- latitude_instalacao: double (nullable = true)
 |-- longitude_instalacao: double (nullable = true)

+------+--------------------+--------+-------------------+--------------------+
|codigo|          instalacao|operador|latitude_instalacao|longitude_instalacao|
+------+--------------------+--------+-------------------+--------------------+
|   COV|    Terminal Coveñas|  Ocensa|  9.409136280745889|  -75.70015297027714|
|   LAG|Estacion La Granjita|  Ocensa|  8.449501151686118|  -75.50994443266501|
|   CAU|     Planta Caucasia|     ODC|  7.976933967792115|  -75.22089861123531|
+------+--------------------+--------+-------------------+--------------------+
only showing top 3 rows



In [0]:
# Caminho do arquivo Delta Lake
file_path = "/mnt/dados/camada_prata_impacto"

# Ler o arquivo Delta Lake
df_impacto = spark.read.format("delta").load(file_path)

df_impacto.printSchema()
df_impacto.show(3)

root
 |-- instalacoes_cod: string (nullable = true)
 |-- terremotos_id: string (nullable = true)
 |-- distancia: double (nullable = true)

+---------------+-------------+------------------+
|instalacoes_cod|terremotos_id|         distancia|
+---------------+-------------+------------------+
|            CHI|   us10001ldx| 165.8097685759375|
|            VAS|   us10001ldx|190.64982815656515|
|            LAB|   us10001ldx| 149.7729764777793|
+---------------+-------------+------------------+
only showing top 3 rows



## Passo 2 - Criar o esquema SQL

_Infelizmente, na versão Community Edition, o Databricks não oferece a possibilidade de criação de databases SQL com garantias de integridade referencial, como uso de chaves primárias e estrangeiras._

In [0]:

dbutils.fs.rm('dbfs:/user/hive/warehouse/impactos', recurse=True)
dbutils.fs.rm('dbfs:/user/hive/warehouse/instalacoes', recurse=True)
dbutils.fs.rm('dbfs:/user/hive/warehouse/terremotos', recurse=True)

Out[5]: True

In [0]:
%sql


DROP TABLE IF EXISTS impactos;
DROP TABLE IF EXISTS instalacoes;
DROP TABLE IF EXISTS terremotos;


CREATE TABLE impactos (
  instalacoes_cod VARCHAR(8),
  terremotos_id VARCHAR(255),
  distancia DOUBLE
);

CREATE TABLE instalacoes (
  codigo VARCHAR(8),
  instalacao VARCHAR(255),
  operador VARCHAR(255),
  latitude_instalacao DOUBLE,
  longitude_instalacao DOUBLE
);

CREATE TABLE terremotos (
  id VARCHAR(255),
  prop_mag DOUBLE,
  prop_url VARCHAR(1024),
  data DATE,
  latitude DOUBLE,
  longitude DOUBLE,
  pais VARCHAR(255)
);





## Passo 3 - Mover dados dos dataframes para o esquema SQL

In [0]:
# Escrevendo o DataFrame df_impactos na tabela impactos
df_impacto.write.mode("overwrite").saveAsTable("impactos")

# Escrevendo o DataFrame df_instalacoes na tabela instalacoes
df_instalacoes.write.mode("overwrite").saveAsTable("instalacoes")

# Escrevendo o DataFrame df_terremotos na tabela terremotos
df_terremotos.write.mode("overwrite").saveAsTable("terremotos")


